In [1]:
%load_ext autoreload
%autoreload 2
from nuplan.common.actor_state.ego_state import EgoState
from nuplan.common.actor_state.state_representation import StateSE2, StateVector2D, TimePoint
from nuplan.common.actor_state.vehicle_parameters import VehicleParameters
from nuplan.planning.simulation.simulation_time_controller.simulation_iteration import SimulationIteration
from nuplan.planning.simulation.trajectory.interpolated_trajectory import InterpolatedTrajectory
from nuplan.planning.simulation.controller.tracker.ilqr_tracker import ILQRTracker
from nuplan.planning.simulation.controller.tracker.ilqr.ilqr_solver import ILQRSolver, ILQRSolverParameters, ILQRWarmStartParameters
from nuplan.planning.simulation.controller.motion_model.kinematic_bicycle import KinematicBicycleModel
from nuplan.planning.simulation.planner.ml_planner.transform_utils import transform_predictions_to_states
from nuplan_extent.planning.training.closed_loop.controllers.two_stage_controller import TwoStageEgoController
from nuplan.planning.scenario_builder.test.mock_abstract_scenario import MockAbstractScenario

import numpy as np
import torch
import time

/home/users/jingyu.qian/horizon-nuplan/nuplan/common/maps/maps_datatypes.py:7: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [2]:
np.set_printoptions(precision=3, linewidth=140)

In [3]:
solver_param = ILQRSolverParameters(
    discretization_time=0.5,
    state_cost_diagonal_entries=[1.0, 1.0, 10.0, 0.0, 0.0],
    input_cost_diagonal_entries=[1.0, 10.0],
    state_trust_region_entries=[1.0, 1.0, 1.0, 1.0, 1.0],
    input_trust_region_entries=[1.0, 1.0],
    max_ilqr_iterations=10,
    convergence_threshold=1e-6,
    max_solve_time=5,
    max_acceleration=3.0,
    max_steering_angle=1.047197,
    max_steering_angle_rate=0.5,
    min_velocity_linearization=0.01
)
warm_start_param = ILQRWarmStartParameters(
    k_velocity_error_feedback=0.5,
    k_steering_angle_error_feedback=0.05,
    lookahead_distance_lateral_error=15.0,
    k_lateral_error=0.1,
    jerk_penalty_warm_start_fit=1e-4,
    curvature_rate_penalty_warm_start_fit=1e-2,
)

In [4]:
# Initialize trackers
ilqr_solver_default = ILQRSolver(solver_param, warm_start_param)
ilqr_tracker_default = ILQRTracker(16, ilqr_solver_default)

In [5]:
scenario = MockAbstractScenario(
    time_step=0.2, initial_velocity=StateVector2D(3.0, 3.0), 
)

In [20]:
past_current_states = [
    scenario.get_ego_state_at_iteration(0),
    scenario.get_ego_state_at_iteration(1),
    scenario.get_ego_state_at_iteration(2),
]

future_state = scenario.get_ego_state_at_iteration(3)
future_state2 = scenario.get_ego_state_at_iteration(4)

In [13]:
# create a fake ego state
vehicle = VehicleParameters(
**{'width': 2.297,
"front_length": 4.049,
"rear_length": 1.127,
"cog_position_from_rear_axle": 1.67,
"height": 1.777,
"wheel_base": 3.089,
"vehicle_name": "pacifica",
"vehicle_type": "gen1"}
)

In [8]:
motion_model = KinematicBicycleModel(vehicle)

In [15]:
# create a fake prediction
trajectory = np.array([
    [ 3.97342634e+00,  4.57113355e-01,  2.49877930e-01, ],
    [ 6.12674379e+00,  1.09291279e+00,  3.52767706e-01, ],
    [ 8.31951332e+00,  1.97754967e+00,  4.13567543e-01, ],
    [ 1.06296530e+01,  3.00840688e+00,  4.53505993e-01, ],
    [ 1.30230131e+01,  4.19286108e+00,  4.81674552e-01, ],
    [ 1.55141821e+01,  5.49916553e+00,  4.97916579e-01, ],
    [ 1.80201054e+01,  6.87377167e+00,  5.07725358e-01, ],
    [ 2.05358086e+01,  8.26056290e+00,  5.15743375e-01, ],
    [ 2.29929905e+01,  9.67429256e+00,  5.23426056e-01, ],
    [ 2.54551449e+01,  1.11441393e+01,  5.30618072e-01, ],
    [ 2.79612312e+01,  1.26187944e+01,  5.37579775e-01, ],
    [ 3.04966621e+01,  1.41300039e+01,  5.44248700e-01, ],
    [ 3.30126915e+01,  1.57168932e+01,  5.51712871e-01, ],
    [ 3.55922203e+01,  1.73329601e+01,  5.59343815e-01, ],
    [ 3.81669350e+01,  1.89929581e+01,  5.67086935e-01, ],
    [ 4.13423423e+01,  2.05123947e+01,  5.74123412e-01, ]
    ]
)

# These are used by the default tracker
traj_as_states = transform_predictions_to_states(trajectory, past_current_states, 8, 0.5, True)
traj_as_traj = InterpolatedTrajectory(traj_as_states)
current_iteration = SimulationIteration(future_state.time_point, 0)

---

In [16]:
controller = TwoStageEgoController(0.5, 8, ilqr_tracker_default, motion_model)

In [18]:
controller.initialize(past_current_states[-1], torch.tensor(0))
controller.last_trajectory = trajectory
controller.update(future_state)
print(controller.states)

deque([<nuplan.common.actor_state.ego_state.EgoState object at 0x7f587c242790>, <nuplan.common.actor_state.ego_state.EgoState object at 0x7f587c242940>], maxlen=10)


In [21]:
controller.update(future_state2)

In [22]:
print(controller.states[-1].rear_axle)

StateSE2(x=2.3999999999999986, y=0.0, heading=0.0)
